# Bootstrap Your Own Latent (BYOL)

In this session we are going to implement Bootstrap Your Own Latent paper (https://arxiv.org/abs/2006.07733).

It uses a MoCo-style training (with asymmetric SiameseNet) but with a L2 loss penalty (it is not a contrastive-base method).

In [ ]:
import os
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from torchvision.io import read_image

import torchvision
import torchvision.models as models
import torchvision.transforms as transforms

from PIL import Image
import copy

In [ ]:
# loss fn
def loss_fn(x, y): #normalizzo i vettori x e y e ritorno la distanza euclidea
    x = F.normalize(x, dim=-1, p=2)
    y = F.normalize(y, dim=-1, p=2)
    return 2 - 2 * (x * y).sum(dim=-1)


class EMA():
    # exponential moving average
    def __init__(self, beta):
        super().__init__()
        self.beta = beta

    def update_average(self, old, new): #prende un parametro della rete target (old) e il corrispettivo della rete online (new)
        if old is None:
            return new
        return old * self.beta + (1 - self.beta) * new

def update_moving_average(ema_updater, ma_model, current_model): #prende l'istanza di EMA (attributo della rete target), la rete target e la rete online
    for current_params, ma_params in zip(current_model.parameters(), ma_model.parameters()): #per ogni corrispondente coppia di parametri delle due reti
        old_weight, up_weight = ma_params.data, current_params.data
        ma_params.data = ema_updater.update_average(old_weight, up_weight) #aggiorna i parametri della rete target

# MLP class for projector and predictor

def MLP(dim, projection_size, hidden_size=4096, sync_batchnorm=None):
    return nn.Sequential(
        nn.Linear(dim, hidden_size),
        nn.BatchNorm1d(hidden_size),
        nn.ReLU(inplace=True),
        nn.Linear(hidden_size, projection_size)
    )


class BYOL(nn.Module):
    def __init__(self, backbone, moving_average_decay = 0.99):
        super().__init__()

        self.target_ema_updater = EMA(moving_average_decay)

        self.online_net = backbone #l'encoder della rete online è la backbone
        self.online_net.fc = nn.Identity() #rimuovo il layer fully connected della rete online
        self.online_projector = MLP(512, 512, 4096) #il projector della rete online è un MLP, che alla fine lascia la dimensione a 512

    def _get_target_encoder(self):
        if self.target_net is None:
            target_net = copy.deepcopy(self.online_net) #la rete target viene inizializzata come deep copy della rete online
            for p in target_net.parameters():
                p.requires_grad = False #disabilito il calcolo del gradiente per ogni peso della rete target, questi infatti si aggiornano tramite EMA
            self.target_net = target_net
        else:
            target_net = self.target_net
        return target_net

    def update_moving_average(self):
        update_moving_average(self.target_ema_updater, self.target_net, self.online_net)

    def forward(self, x1, x2):

        images = torch.cat((x1, x2), dim = 0) #concateno i batch delle due viste

        # con self.online_net(images) passo le immagini della rete online nella backbone, e il risultato lo passo al projector ottenendo le relative proiezioni
        online_projections = self.online_projector(self.online_net(images))


        online_pred_one, online_pred_two = online_projections.chunk(2, dim = 0) #le proiezioni le splitto in due tensori, uno per vista

        with torch.no_grad():
            target_net = self._get_target_encoder()

            target_projections = target_net(images) #ottengo le proiezioni delle immagini embedded
            target_projections = target_projections.detach()
            target_proj_one, target_proj_two = target_projections.chunk(2, dim = 0) #anche qui splitto le proiezioni in due tensori, uno per vista

        loss_one = loss_fn(online_pred_one, target_proj_two.detach()) #loss calcolata usando la prima vista nella rete online e la seconda vista nella rete target
        loss_two = loss_fn(online_pred_two, target_proj_one.detach()) #loss calcolata usando la seconda vista nella rete online e la prima vista nella rete target

        #sommo le due loss e ritorno la media
        loss = loss_one + loss_two
        return loss.mean()

## Exercise 0

Study the above code.
- Where is the EMA updates? #dove interviene nel training, cosa e come freezzi il gradiente?
- Why it computes both loss_one and loss_two values? -> La loss è simmetrica, è somma di due modi diversi di dare le viste all'encoder. In Sto confrontanfdo le l'uscita della rete online su una trasformazione con l'uscita della rete target su un'altra trasformazione della stessa immagine (simil-positive)

Vogliamo che le reti imparino l'una dall'altra quindi la loss deve essere effettuata tra view diverse delle 2 reti - calcolo a croce.

## Exercise 1

Write the training loop for moco-style training as used in BYOL.
Use the Dataset which creates the two augmented views for each image and the Siamese Network from the past lab session [1](https://colab.research.google.com/drive/1NJwAFbRiD4MdwWf__6P2Lm0xYk_DNdVu?usp=sharing) and [2](https://colab.research.google.com/drive/1AMkh0q8L5nJScx7v6cMWoK336zqOqDY6?usp=sharing).